In [213]:
#import modules

import pandas as pd
import numpy as np

from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

In [214]:
#import and clean the data

earthquake_df=pd.read_csv('Earthquake.csv',delimiter=';')
earthquake_df['Latitude']=earthquake_df['Latitude'].astype('float')
earthquake_df['Longitude']=earthquake_df['Longitude'].astype('float')
earthquake_df['Depth']=earthquake_df['Depth'].astype('int64')
earthquake_df['Magnitude']=earthquake_df['Magnitude'].astype('int64')
earthquake_df=earthquake_df.reset_index()
earthquake_df=earthquake_df.drop('index',axis=1)
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       23412 non-null  object 
 1   Year       23412 non-null  object 
 2   Time       23412 non-null  object 
 3   Latitude   23412 non-null  float64
 4   Longitude  23412 non-null  float64
 5   Type       23412 non-null  object 
 6   Depth      23412 non-null  int64  
 7   Magnitude  23412 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 1.4+ MB


In [215]:
#Bokeh maps are in mercator. Convert lat lon fields to mercator units for plotting

def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df=wgs84_to_web_mercator(earthquake_df,'Longitude','Latitude')

In [216]:
#Establishing a zoom scale for the map. The scale variable will also determine proportions for hexbins and bubble maps so that everything looks visually appealing. 

scale=2000
x=earthquake_df['x']
y=earthquake_df['y']

#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.

x_min=int(x.mean() - (scale * 350))
x_max=int(x.mean() + (scale * 350))
y_min=int(y.mean() - (scale * 350))
y_max=int(y.mean() + (scale * 350))

#Defining the map tiles to use. I use OSM, but you can also use ESRI images or google street maps.

tile_provider=get_provider(OSM)

#Establish the bokeh plot object and add the map tile as an underlay. Hide x and y axis.

plot=figure(
    title='Map of Earthquake',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator'
    )

plot.grid.visible=True

map=plot.add_tile(tile_provider)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False

#If in Jupyter, use the output_notebook() method to display the plot in-line. If not, you can use output_file() or another method to save your map. 
output_notebook()

Loading BokehJS ...

In [217]:
#function takes scale (defined above), the initialized plot object, and the converted dataframe with mercator coordinates to create a hexbin map

def hex_map(plot,df, scale,leg_label='Hexbin Heatmap'):
  r,bins=plot.hexbin(x,y,size=scale*10,hover_color='pink',hover_alpha=0.5,legend_label=leg_label)
  hex_hover = HoverTool(tooltips=[('count','@c')],mode='mouse',point_policy='follow_mouse',renderers=[r])
  hex_hover.renderers.append(r)
  plot.tools.append(hex_hover)
  
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

In [218]:
#function takes a column to determine radius and the dataframe with converted mercator coordinates to create a bubble map. 
def bubble_map(plot,df,radius_col,lon,lat,mag,scale,color='orange',leg_label='Bubble Map'):
  radius=[]
  for i in df[radius_col]:
    radius.append(i*scale)
  
  df['radius']=radius
    
  source=ColumnDataSource(df)
  c=plot.circle(x='x',y='y',color=color,source=source,size=1,fill_alpha=0.4,radius='radius',legend_label=leg_label,hover_color='red')

  tip_label='@'+radius_col
  mag_label='@'+mag
  lat_label='@'+lat
  lon_label='@'+lon

  circle_hover = HoverTool(tooltips=[(radius_col,tip_label),('Magnitude:',mag_label),('Lat:',lat_label),('Lon:',lon_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
  circle_hover.renderers.append(c)
  plot.tools.append(circle_hover)

#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

In [219]:
#Create the hexbin map
hex_map(plot=plot,
        df=earthquake_df, 
        scale=scale,
        leg_label='Number of Events Earthquake')

#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.
bubble_map(plot=plot,
           df=earthquake_df,
           radius_col='Depth', 
           leg_label='Depth of the Earthquake',
           lon='y',
           lat='x',
           mag='Magnitude',
           scale=scale)

#let's see our handiwork!
show(plot)